# 协程  coroutine

In [ ]:
协程不是计算机体统，由人为创造。
协程也可被称为微线程，是一种用户态内上下文切换技术，简而言之，其实就是通过线程实现代码块相互切换执行

In [1]:
def func1():
    print(1)
    print(2)
def func2():
    print(3)
    print(4)

In [3]:
func1()
func2()

1
2
3
4


实现协程的几种方法：
>greenlet 第三方库  
>yield 关键字  
>ayncio  
>async,await关键字

# Greenlet 实现协程

In [6]:
from greenlet import greenlet

In [7]:
def func1():
    print(1)
    gr2.switch()
    print(2)
    gr2.switch()

def func2():
    print(3)
    gr1.switch()
    print(4)

In [8]:
gr1 = greenlet(func1)
gr2 = greenlet(func2)

In [9]:
# trigger func1
gr1.switch()

1
3
2
4


# 1.2 yield 关键字

In [10]:
def func1():
    yield 1
    yield from func2()
    yield 2
    
def func2():
    yield 3
    yield 4
    

In [11]:
f1 = func1()
for item in f1:
    print(item)

1
3
4
2


# 1.3 asyncio

In [15]:
import asyncio

@asyncio.coroutine
def func1():
    print(1)
    yield from asyncio.sleep(2) #遇到IO耗时操作，自动切换到tasks中的其他任务
    print(2)
    
@asyncio.coroutine
def func2():
    print(3)
    yield from asyncio.sleep(2) #遇到IO耗时操作，自动切换到tasks中的其他任务
    print(4)

<ipython-input-15-6cf7672c33f1>:4: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def func1():
<ipython-input-15-6cf7672c33f1>:10: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def func2():


In [16]:
tasks = { 
    asyncio.ensure_future( func1()),
    asyncio.ensure_future( func2())
}

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

RuntimeError: This event loop is already running

1
3
4
2


# 1.4 async & await 关键字

In [17]:
# 3.7 以后  用async await
import asyncio

# @asyncio.coroutine
async def func1():
    print(1)
#     yield from asyncio.sleep(2) #遇到IO耗时操作，自动切换到tasks中的其他任务
    await asyncio.sleep(2)
    print(2)
    
# @asyncio.coroutine
async def func2():
    print(3)
#     yield from asyncio.sleep(2) #遇到IO耗时操作，自动切换到tasks中的其他任务
    await asyncio.sleep(2)
    print(4)

In [18]:
tasks = { 
    asyncio.ensure_future( func1()),
    asyncio.ensure_future( func2())
}

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

RuntimeError: This event loop is already running

1
3
2
4


# 协程意义

遇到耗时IO时，不会傻傻的等，可以去做其他事

# 3 异步编程

## 3.1 事件循环

理解为一个死循环，去检测并执行某些代码

In [ ]:
#伪代码
任务列表 =[task1,task2,task3]
while True:
    可执行的任务列表，已完成的任务列表 = 去任务列表中检查所有的任务，
    将‘可执行’和‘已完成’的任务返回
    for 就绪任务 in 可执行的任务列表:
        在任务列表中移除 已完成的任务
    
    如果 任务列表中的任务都已完成  则终止循环

## 3.2 快速上手

协程函数 async def  
协程对象，执行 协程函数()得到的协程对象

In [30]:
async def func():
    print('hello async')


In [25]:
result = func()

如果想要运行协程函数内部代码，必须要把协程对象交给事件循环来处理

In [27]:
# loop = asyncio.get_event_loop()
# loop.run_until_complete( result )

In [29]:
asyncio.run(result)  #py3.7 or later 

RuntimeError: asyncio.run() cannot be called from a running event loop

## 3.3 await

await + 可等待的对象 （协程对象，future，taks对象->io等待）

In [1]:
import asyncio

async def func():
    print('come and play')
    response = await asyncio.sleep()
    print('end',response)

In [ ]:
asyncio.run( func())

In [ ]:
import asyncio

async def others():
    print('come and play')
    await asyncio.sleep(2)
    print('end')
    return 'value'

async def func():
    print('exec coroutine code')
    #遇到IO操作挂起当前协程（任务），等io操作完成之后再继续往下执行，
    # 当前协程挂起时，事件循环可以去执行其他协程（任务）
    response = await others()
    print('IO request finish, result is:',response)
asyncio.run(func())

In [ ]:
import asyncio

async def others():
    print('come and play')
    await asyncio.sleep(2)
    print('end')
    return 'value'

async def func():
    print('exec coroutine code')
    response = await others()
    print('IO request finish, result is:',response)
    
asyncio.run(func())

await就是等待对象的值得到之后再继续往下走

## 3.4 task 对象

In [ ]:
tasks are used to schedule coroutines concurrently.  
when a coroutine is wrapped into a task with functions like asyncio.create_tasks()  
the coroutine is automatically scheduled 

In [ ]:
import asyncio

async def others():
    print('come and play')
    await asyncio.sleep(2)
    print('end')
    return 'value'

async def main():
    print('main start')
    
    task_list = [
        asyncio.create_task( func(),name='n1'),
        asyncio.create_task( func(),name='n2')
    ]
    
    print('main end')
    
    done,pending = await asyncio.wait(task_list,timeout=None)
    print(done)

asyncio.run( main())

In [ ]:
import asyncio

async def others():
    print('come and play')
    await asyncio.sleep(2)
    print('end')
    return 'value'

taks_list = [func(),func()]

done,pending=asyncio.run(asyncio.wait(task_list))

## 3.5 Future 对象

## 3.6 concurrent.futures.Future 对象

## 3.7 异步迭代器

## 3.8 异步上下文管理器

## 4. unloop

In [ ]:
asyncio的事件循环的替代方案。事件循环>默认的asyncio的事件循环

In [ ]:
import asyncio
import uvloop
asyncio.set_event_loop_policy(uvloop.EventloopPolicy())
#其他用法照常
#内部的事件循环自动会变为uvloop


# 实战案例

## 5.1 异步redis

在使用python代码操作redis时，链接、操作，断开都是网络io

In [ ]:
import asyncio
import aioredis


## 5.2 异步mysql

## 5.3 FastAPI框架

## 5.4 爬虫

In [ ]:
import aiohttp
import asyncio

async def fetch(session,url):
    print("send request",url)
    async with session.get(url,verify_ssl=False) as response:
        text = await response.text()
        print("got data",url,len(text))

async def main():
    async with aiohttp.ClientSession() as session:
        url_list =  [
            'https://python.org',
            'https://www.baidu.com',
            'https://www.pythonav.com'
        ]
    tasks = [asyncio.create_task(fetch(session,url)) for url in url_list]
    
    done,pending = await asyncio.wait(tasks)
    
if __name__ == '__main___':
    asyncio.run(main())